In [1]:
import pandas as pd
from datetime import datetime as dt
from binance.client import Client
from sqlalchemy import create_engine
from secrets import secrets

In [2]:
BINANCE_API_KEY = secrets['BINANCE_API_KEY']
BINANCE_SECRET_KEY = secrets['BINANCE_SECRET_KEY']
DATABASE_URL = secrets['DATABASE_URL']

BRIDGE = 'USDT'
TRANS_FEE = 0.0075
CHAT_ID = 123

In [3]:
df_invest = pd.DataFrame(columns=['Id', 'Date', 'Amount'])
df_buy = pd.DataFrame(columns=['Id', 'Date', 'Coin', 'UsdAmount', 'CoinAmount'])
df_sell = pd.DataFrame(columns=['Id', 'Date', 'Coin', 'CoinAmount', 'UsdAmount'])

In [4]:
# df_pf = pd.DataFrame(columns=['Id', 'Coin', 'Amount'])
df_trans = pd.DataFrame(columns=['Id', 'Date', 'Action', 'USD', 'Coin', 'Amount', 'Price'])

### Invest

In [5]:
def modify_portfolio(chat_id, coin, amount):
    curr_amount = df_pf.loc[(df_pf['Id'] == chat_id) & (df_pf['Coin'] == coin), 'Amount']
    if len(curr_amount) > 0:
        df_pf.loc[(df_pf['Id'] == chat_id) & (df_pf['Coin'] == coin), 'Amount'] += amount
    else:
        df_pf.loc[len(df_pf)] = [chat_id, coin, amount]

In [6]:
def invest(chat_id, amount):
    df_trans.loc[len(df_trans)] = [chat_id, dt.now(), 'IN', amount, None, None, None]
    # modify_portfolio(chat_id, 'USD', amount)

### Buy

In [7]:
def check_price(coin):
    symbol = coin + BRIDGE
    client = Client(BINANCE_API_KEY, BINANCE_SECRET_KEY)
    t = client.get_ticker(symbol=symbol)
    return float(t['lastPrice'])

In [8]:
def buy(chat_id, coin, amount):
    df_temp = df_trans.loc[df_trans['Id'] == chat_id]
    curr_amount = df_trans['USD'].sum()
    if curr_amount < amount:
        return -1
    price = check_price(coin)
    new_amount = (amount / price) * (1 - TRANS_FEE)
    df_trans.loc[len(df_trans)] = [chat_id, dt.now(), 'BUY', -1 * amount, coin, new_amount, price]
    # df_pf.loc[(df_pf['Id'] == chat_id) & (df_pf['Coin'] == 'USD'), 'Amount'] -= amount
    # modify_portfolio(chat_id, coin, new_amount)
    return 0

### Sell

In [9]:
def sell(chat_id, coin, amount):
    df_temp = df_trans.loc[(df_trans['Id'] == chat_id) & (df_trans['Coin'] == coin)]
    curr_amount = df_trans['Amount'].sum()
    if curr_amount < amount:
        return -1
    price = check_price(coin)
    new_amount = amount * price * (1 - TRANS_FEE)
    df_trans.loc[len(df_trans)] = [chat_id, dt.now(), 'SELL', new_amount, coin, -1 * amount, price]
    # df_pf.loc[(df_pf['Id'] == chat_id) & (df_pf['Coin'] == coin), 'Amount'] -= amount
    # modify_portfolio(chat_id, 'USD', new_amount)
    return 0

### Test

In [10]:
invest(CHAT_ID, 1000)
buy(CHAT_ID, 'BTC', 1000)
sell(CHAT_ID, 'BTC', 0.01)
buy(CHAT_ID, 'BNB', 500)
sell(CHAT_ID, 'BNB', 0.5)

0

In [11]:
df_trans

,Id,Date,Action,USD,Coin,Amount,Price
0,123,2021-04-12 00:00:31.111637,IN,1000,None,None,None
1,123,2021-04-12 00:00:31.966630,BUY,-1000,BTC,0.0166084,59758.8
2,123,2021-04-12 00:00:32.785623,SELL,593.108,BTC,-0.01,59759
3,123,2021-04-12 00:00:33.786456,BUY,-500,BNB,1.00458,493.988
4,123,2021-04-12 00:00:34.569564,SELL,245.148,BNB,-0.5,494


In [12]:
df_trans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Id      5 non-null      object        
 1   Date    5 non-null      datetime64[ns]
 2   Action  5 non-null      object        
 3   USD     5 non-null      object        
 4   Coin    4 non-null      object        
 5   Amount  4 non-null      object        
 6   Price   4 non-null      object        
dtypes: datetime64[ns](1), object(6)
memory usage: 320.0+ bytes


### Portfolio

In [13]:
def portfolio(chat_id):
    df_temp = df_trans.loc[df_trans['Id'] == chat_id]
    initial = df_temp.loc[df_temp['Action'] == 'IN', 'USD'].sum()
    curr_usd = df_temp['USD'].sum()
    df_coins = df_temp.groupby('Coin')['Amount'].sum().reset_index()
    df_coins = df_temp.groupby('Coin')['Amount'].sum().reset_index()
    df_coins['Price'] = df_coins['Coin'].apply(lambda x: check_price(x))
    df_coins['Value'] = df_coins['Amount'] * df_coins['Price']
    curr_val = curr_usd + df_coins['Value'].sum()
    profit = curr_val - initial
    percent = profit / initial
    return curr_val, profit, percent

In [14]:
portfolio(CHAT_ID)

(982.4095348027399, -17.590465197260073, -0.017590465197260072)

In [16]:
df_coins = df_trans.groupby('Coin')['Amount'].sum().reset_index()
df_coins['Price'] = df_coins['Coin'].apply(lambda x: check_price(x))
df_coins['Value'] = df_coins['Amount'] * df_coins['Price']
df_coins.head()

,Coin,Amount,Price,Value
0,BNB,0.504580,493.3548,248.936902
1,BTC,0.006608,59749.3300,394.850082


### To SQL

In [17]:
engine = create_engine(DATABASE_URL)
# df_pf.to_sql('Portfolio', engine, index=False, if_exists='replace')
df_trans.to_sql('Transaction', engine, index=False, if_exists='replace')
engine.dispose()